In [26]:
import pandas_datareader.data as web
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import warnings
import pandas as pd
from pandas import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
register_matplotlib_converters()
from time import time
warnings.filterwarnings("ignore")
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf
from datetime import date
from datetime import datetime
import yfinance as yf
import plotly.graph_objects as go
from random import *

In [27]:
#Choose Trading Platform
broker_auswahl = pd.read_csv('Trading_Broker.csv')

broker = str(input("Gewünschter Online-Broker: "))
print(broker)

i = broker_auswahl[broker_auswahl['Anbieter'] == broker].index
print(i[0])

print(broker_auswahl.at[i[0], "Gebühr"])
order_fee = broker_auswahl.at[i[0], "Gebühr"]
min_ = broker_auswahl.at[i[0], "Minimum"]
max_= broker_auswahl.at[i[0], "Maximum"]

IG
1
0.05


In [28]:
#Funktion zur Berechnung der Trading-Gebühren
def trading_fee(endowment):
    fee = endowment*(order_fee/100)
    
    #Prüfung, ob Mindest- oder Maximalgebühr des ausgewählten Brokers greift
    if fee <= min_:
        fee = min_
        
    if max_.isnumeric():
        if fee >= max_:
            fee = max_
        
    return fee

In [33]:
#Nutzerentscheidung, ob Short-Selling aktiviert werden soll -> Eingabe: "Yes" oder "No"
short = False
answer = str(input("Include short-selling?: "))
print(answer)
if answer == "Yes":
    short = True

no


In [30]:
#Funktion zum Download der relevanten Daten
def get_data(asset, start, end):
    ticker = asset
    data = yf.download(ticker, start=start, end=end)

    # calculate returns and format df
    data["Returns"] = 100 * data["Adj Close"].pct_change()
    data = data.dropna()
    data["Date"] = pd.to_datetime(data.index)
#Funktion zum Download der relevanten Daten
    data.reset_index(drop=True,inplace=True)
    return data

In [31]:
#In dieser Funktion findet das eigentliche Trading und die zugehörigen Berechnungen statt
def trade(data, trade_day_index, endowment, profit, cash):
    action = []
    real_returns = []
    forecasts = []
    tax_allowance = 801

    # forecast ist random
    forecast = randint(-1,1)
    real_return = data["Returns"][trade_day_index]
    
    prev_endowment = endowment
        
    
    if forecast >= 0.0 and cash == True:  #BUY 
        endowment = endowment*(real_return/100+1) - trading_fee(endowment) # Berücksichtigung von trading fees
        action.append("BUY")
        cash = False

    elif forecast >= 0.0 and cash == False:  #HOLD
        endowment = endowment*(real_return/100+1)
        action.append("HOLD")
        cash = False

    elif forecast <= 0.0 and cash == True:
        if short == True: #short-selling
            endowment = endowment - (real_return/100)*endowment - trading_fee(endowment) # Berücksichtigung von trading fees
            gain = endowment - prev_endowment
            if isinstance(gain, float) == False: #die isinstance-Abfragen (auch die weiter unten) werden nur gemacht weil es bei den folgenden if-Abfragen sonst zu Fehlermeldungen wegen dem zugehörigen Index kommt
                gain = gain.item()
            if gain > 0.0:
                profit = profit + gain
                #Falls der Gewinn 801 GE noch nicht übersteigt, wird der volle Gewinn aus diesem Trade angerechnet. Andernfalls werden die Kapitalertragssteuern + Soli (und ggf. Kirchensteuer) abgezogen
                if profit < 801:
                    endowment = endowment + gain
                else:
                    endowment = endowment + (1-0.26375)*gain
            action.append("SHORT")
            cash == True
        else: #HOLD
            endowment = endowment
            action.append("HOLD")
            cash = True

    elif forecast <= 0.0 and cash == False:  #SELL
        endowment = endowment - trading_fee(endowment) # Berücksichtigung von trading fees
        gain = endowment - prev_endowment
        if isinstance(gain, float) == False:
            gain = gain.item()
            #Berücksichtigung steuerlicher Freibetrag, falls drüber, Abzug Kapitalertragssteuer + Kirchensteuer vom Gewinn aus Trade
        if gain >= 0:
            profit = profit + gain
            if profit < 801:
                endowment = endowment + gain
            else:
                endowment = endowment + (1-0.26375)*gain
        action.append("SELL")
        cash = True
        
    if isinstance(endowment, float) == False:
        endowment = endowment.item()
    
    
    return data, endowment, action, profit, cash

In [34]:
today = date.today()
#profit = 0 #evtl. hier richtig
#stocks = ["DTG.DE","ROTH.PA", "DBK.DE", "WMT", "BA", "BRK-B", "CMG", "NFC.F", "MSFT", "WBD", "JNJ", "BP", "DIS", "BAC", "VOLV-B.ST", "AMZN","AAPL","BMW.DE", "SBUX", "GS", "RHM.DE", "AHLA.DE", "BEI.DE", "CCC3.F", "BAYN.DE", "BOSCHLTD.BO", "LHA.DE", "HLT", "TUI1.DE", "MCD", "MC.PA", "GOOG", "MSCI", "MAR", "GE", "TSLA", "LUV", "ATVI", "V", "RWE.DE"]
stocks = ["ROTH.PA", "DBK.DE", "WMT", "BA", "BRK-B", "CMG", "NFC.F", "MSFT", "WBD", "JNJ", "BP", "DIS", "BAC", "AMZN","AAPL","BMW.DE", "SBUX", "GS", "RHM.DE", "AHLA.DE", "BEI.DE", "CCC3.F", "BAYN.DE", "BOSCHLTD.BO", "LHA.DE", "HLT", "TUI1.DE", "MCD", "MC.PA", "GOOG", "MSCI", "MAR", "GE", "TSLA", "LUV", "ATVI", "V", "RWE.DE"]
#stocks = ["AAPL"]
endowment_after_trade = {}
ergebnis = []
number_stocks = len(stocks)
counter = 0
all_actions = {} # Kaufenscheidungen pro Aktie pro Tag

for stock in stocks:
    profit = 0
    start = datetime(2020, 12, 30) # Erster Prognose Tage
    start = start.date()
    trade_day = datetime(2021, 1, 4) # Erster Handelstag
    trade_day = trade_day.date()
    trade_day_index = 1
    end = datetime(2021, 4, 1) # Letzter Handelstag
    end = end.date()
    print(stock)
    endowment = 10000.0 #Wie viel Geld soll in jeweilige Aktie investiert werden
    start_endowment = endowment
    cash = True

    actions = []

    while trade_day != end: #Eventuell fehlermeldung weil in end zeit mit drin ist
    
        print(trade_day)
    
        if np.is_busday(trade_day) == True:
            data = get_data(stock, start, end+timedelta(days=5))            
            data, endowment, action, profit, cash = trade(data, trade_day_index, endowment, profit, cash)
            actions.append(action)
            print(action)
            trade_day_index += 1
            
            #if endowment <= start_endowment*0.96:
                #break
            
        else:
            print("No trading day (Weekend etc.)")
    
        
        trade_day = trade_day + timedelta(days=1)
        print(endowment)
        
    
    print(endowment)
    ergebnis.append(endowment)
    endowment_after_trade[stock]=endowment
    all_actions[stock] = actions
    


print(endowment_after_trade)
print(all_actions)
print(ergebnis)



ROTH.PA
2021-01-04
[*********************100%***********************]  1 of 1 completed
['HOLD']
10000.0
2021-01-05
[*********************100%***********************]  1 of 1 completed
['BUY']
9824.868010148328
2021-01-06
[*********************100%***********************]  1 of 1 completed
['HOLD']
10070.48939383374
2021-01-07
[*********************100%***********************]  1 of 1 completed
['HOLD']
10146.06533189489
2021-01-08
[*********************100%***********************]  1 of 1 completed
['HOLD']
10278.323619212302
2021-01-09
No trading day (Weekend etc.)
10278.323619212302
2021-01-10
No trading day (Weekend etc.)
10278.323619212302
2021-01-11
[*********************100%***********************]  1 of 1 completed
['HOLD']
10202.747681151153
2021-01-12
[*********************100%***********************]  1 of 1 completed
['HOLD']
10335.005968468566
2021-01-13
[*********************100%***********************]  1 of 1 completed
['HOLD']
10316.111588242877
2021-01-14
[***********

In [36]:
m1s = [11532.714336220097, 10294.674812990539, 10686.444569742107, 9963.515696067172, 10452.380990161904, 11517.441609508778, 8656.948077500485, 9172.525480017912, 41537.18938183335, 10282.816022968567, 11598.867586697057, 9386.493309862874, 13789.343827138831, 9551.72127566436, 10976.066250142576, 10277.99908773604, 8624.316705675277, 12133.594986841594, 10459.473891861146, 7962.176771167154, 10497.390748035743, 10459.162373912664, 8512.952949210096, 10466.227560501211, 14143.198431891069, 10638.949702360787, 8530.878156999224, 8454.183289058068, 10067.238771630704, 10721.307208936994, 11518.620228675356, 13730.087302130387, 12379.431679190538, 8845.551102281986, 12143.567558294602, 11754.219977688603, 9485.600585129661, 12669.405956410938]

m1 = [111863.68510399672, 100075.16570575246, 94331.46951745042, 102390.78710845554, 106368.94757999577, 97977.04076640714, 90156.5912068854, 97407.48279837582, 215097.56742285742, 99169.82876745313, 111628.3272058837, 96081.17317223901, 119569.53340007737, 92848.44839761968, 96330.3112797185, 106916.37904738932, 90006.03215233682, 116267.0208065894, 99336.69007010228, 94097.0600013033, 92740.684785675, 99638.6179259321, 92429.23939557235, 97650.49828690877, 117618.67465408983, 107085.95788569635, 89259.82896660492, 90830.21066474002, 100047.62183625554, 110818.55704465004, 101833.24652486613, 115322.69307464128, 114804.07882346062, 88122.46643859407, 118106.42517853934, 104508.73096226908, 94104.58333029566, 103920.38961715899]
m2s = [125352.32419698616, 120648.67497429803, 99263.84899686198, 121015.11578277088, 105698.08553308353, 99350.59339645185, 91096.82677914905, 97848.11349428899, 390264.08398468414, 102740.19925219826, 138471.63263891608, 75839.84739734157, 133343.00646542278, 86160.02980118192, 113258.93776657619, 99298.86348857378, 103798.67542660836, 135686.9980057206, 107031.76400324925, 92592.79100742024, 100394.39380423354, 86819.53580825256, 84209.30630841991, 108658.81017438794, 100487.20570834228, 113817.908101219, 173492.55336093283, 94529.10733416278, 92800.04490747912, 110718.65843312627, 90801.12947882971, 111948.25815615075, 83747.71099735271, 106158.57792989617, 112420.70251481743, 108214.27688179688, 106709.1076125476, 133499.99233039282]
m2 = [120242.58765464641, 109196.32852269016, 95536.85645357493, 100792.68420372451, 105818.07071487952, 102678.35804489872, 92798.55461864184, 101053.38857770253, 212403.09172255197, 105288.25417967836, 115030.90129112272, 87814.19149844158, 120706.64243462081, 89025.97503048435, 96162.8913285348, 105700.03297904566, 99591.38690586622, 123419.82257613156, 97251.13247759851, 92639.41947653562, 96763.51464754822, 93918.51653602853, 93864.50058005659, 101487.39162959288, 104824.42148899529, 101672.1646850414, 121083.05694083523, 97627.19851780288, 97628.80486509236, 113704.88363233114, 93096.57870204159, 106152.88434473649, 101047.8908559892, 95931.69269268622, 117424.71738816942, 96998.38732583333, 98121.2673331716, 108234.06674642414]
rdms = [121800.35295426556, 94671.24344805379, 95178.27837696018, 122050.54593140131, 102361.45364988866, 107525.72625362405, 105789.60532266999, 104310.92170423716, 148682.45547387414, 102060.02478652538, 129348.0511921682, 113644.26822834101, 139751.9145455319, 110295.17927344311, 95960.97118826682, 139592.94104794037, 105031.73365610787, 122191.16349749293, 103005.93585439697, 94364.01713597451, 95864.71447299895, 115227.77989559852, 109691.66804057366, 120312.67428068326, 84543.45272911944, 147154.54259006082, 142039.21873165158, 111061.572546272, 102019.63106907533, 113020.25171630351, 88920.85823249578, 99297.3961104258, 99431.52610639784, 112213.69924773886, 141643.21585527758, 102785.503793058, 90637.54846752023, 110658.50795404718]
rdm = [109253.01524803966, 110472.65707632569, 103690.89095660899, 91577.33642289658, 102317.81988349889, 100745.55689473695, 82268.39269077685, 111974.57171322852, 148710.16707949634, 102848.90194090059, 115977.89216717666, 86460.65707908632, 120633.19599973244, 98874.71733643782, 112418.76954685131, 121078.89940092494, 113692.0332443493, 113998.3765035335, 111033.9375652207, 107570.68678033183, 90556.39409119116, 100649.34896862818, 105556.54260201838, 109902.91351749614, 117465.2952334984, 103382.97597092738, 111221.05538778503, 108930.131977458, 106401.6089837585, 114700.35889525742, 101049.8159354722, 93263.8589276754, 107280.18273410153, 121530.69737637963, 104471.45024812793, 107610.11680871648, 90693.293997452, 88993.40881523506]

print(sum(m1s)/len(m1s))
print(sum(m1)/len(m1))
print(sum(m2s)/len(m2))
print(sum(m2)/len(m2))
print(sum(rdms)/len(rdms))
print(sum(rdm)/len(rdm))

11417.754690845697
104651.6328133379
114689.14979563485
105598.22393694073
111687.90908843318
106559.41910529824
